La clase Atomos de ASE nos permite crear un sistema molecular para hacer un estudio de simulación computacional de ellos. 

Vamos a realizar ahora en clase el tutorial de ASE que se llama "Introduction: Nitrogen on copper". [link text](https://wiki.fysik.dtu.dk/ase/gettingstarted/surface.html) 

En el siguiente modulo van a encontrar una informacion extensa del modulo de Atomos que pueden leer poco a poco para cubrir todos los detalles relacionados con el modulo de ASE. [link ASE ](//wiki.fysik.dtu.dk/ase/ase/atoms.html#module-ase).

Por ahora empecemos ejecutando la celda siguiente que les permite importar el codigo ASE y GPAW.Debe tomar unos 30 segundos solamente en ejecutarse.


In [ ]:
%%capture

!apt install python3-mpi4py cython3 libxc-dev gpaw-data
!pip -q install gpaw
!pip -q install nglview

Volvamos al tutorial y empecemos con las lineas que están descritas paso a paso empezando por la sección Atomos.

In [ ]:
from ase import Atoms

d = 1.10

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

In [ ]:
from  ase.visualize import  view 
from google.colab import output
output.enable_custom_widget_manager()

view(molecule, viewer='ngl')

Ahora puede escoger diferentes propiedades del objeto Atoms, modifiquelas y verifique con el visualizador que el objeto cambia de la manera esperada. En la página del módulo que les indiqué mas arriba encuentran listadas todas las propiedades de Atoms asi como metodos que empiezan por "get_" que les permiten monitorear que las propiedades que cambiaron realmente quedaron modificadas. Sugerencia: cambie el tipo de atomos, las posiciones y el momento magnetico pero puede intentar con todas las propiedades que le parezcan importantes.

Ejecute ahora el resto del tutorial. Primero agreguemos un calculador simple llamado EMT y obtengamos la energia potencial de una superficie de cobre y de la molecula.

In [ ]:
#You can also build crystals using, for example, the lattice module which 
#returns Atoms objects corresponding to common crystal structures. Let us make 
#a Cu (111) surface:
from ase.build import fcc111

slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

#In this overview we use the effective medium theory (EMT) calculator, 
#as it is very fast and hence useful for getting started.
#We can attach a calculator to the previously created Atoms objects:
from ase.calculators.emt import EMT

slab.calc = EMT()

molecule.calc = EMT()

#and use it to calculate the total energies for the systems by using the 
#get_potential_energy() method from the Atoms class:
e_slab = slab.get_potential_energy()

e_N2 = molecule.get_potential_energy()

Vamos a agregar la molecula encima de la superficie y relajar el conjunto

In [ ]:
from ase.build import add_adsorbate

h = 1.85

add_adsorbate(slab, molecule, h, 'ontop')

#In order to speed up the relaxation, let us keep the Cu atoms fixed in the slab 
#by using FixAtoms from the constraints module. Only the N2 molecule is then allowed 
#to relax to the equilibrium structure:

from ase.constraints import FixAtoms

constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])

slab.set_constraint(constraint)

#Now attach the QuasiNewton minimizer to the system and save the trajectory file. 
#Run the minimizer with the convergence criteria that the force on all atoms 
#should be less than some fmax:

from ase.optimize import QuasiNewton

dyn = QuasiNewton(slab, trajectory='N2Cu.traj')

dyn.run(fmax=0.05)


In [ ]:
from ase.io import read
relaxed = read("N2Cu.traj@-1")
view(relaxed, viewer="ngl")